In this lab, we will ask any one LLM to give us a chanllenging question to test the intelligence of various LLM's.
then we will ask the same question to all LLM's configured and print their responses.
We will then ask ChatGPT to look at all responses from multiple LLM's and give us a ranking. 
Should be a fun experiment.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [16]:
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]
messages

In [11]:
response = openai_python_client.chat.completions.create(model='gpt-4o-mini', messages=messages)

In [ ]:
question = response.choices[0].message.content
question

In [13]:
# Ok we have our question now. We will query all LLM's now.
messages = [{"role": "user", "content": question}]


In [14]:
competitors = [] # to store the model name
answers = [] # to store the response from each LLM

In [ ]:
# 1. Claude LLM from Anthropic
# Remember i said openai client api can be used by all LLM's. Somehow Anthropic does not use it, so we have to use it's own API
model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()

# Its claude.messages.create(with an extra max_tokens argument) instead of openai.chat.completions.create()
# max_tokens is used to limit the length of the output and to control cost.
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)

answer = response.content[0].text #

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# 2. ChatGPT from OpenAI
model_name = "gpt-4o-mini"

openai_python_client = OpenAI()
response = openai_python_client.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


In [ ]:
# 3. Google Gemini from Google
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# Deepseek next
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# groq by Meta
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

In [ ]:
!ollama pull llama3.2:latest

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2:latest"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n{answer}\n\n"
print(together)

In [ ]:
question

In [ ]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each competitor has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""
print(judge)


In [ ]:
judge_messages = [{"role": "user", "content": judge}]

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!
import json
results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")